In [1]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

import pandas as pd

In [2]:
columnas = ['departamento','municipio','estrato','etapa','maiz_c',
            'frijol_c','arroz_c','otros_usos','maiz_p','frijol_p','arroz_p']


In [3]:
df1 = pd.read_excel('base17.xlsx', skiprows=5, usecols='C,D,E,F,K,L,M,N,O,P,Q')
df2 = pd.read_excel('base19.xlsx', skiprows=5, usecols='C,D,E,F,K,L,M,N,O,P,Q')
df3 = pd.read_excel('base20.xlsx', skiprows=5, usecols='C,D,E,G,O,P,Q,R,S,T,U')
df1.columns = columnas
df2.columns = columnas
df3.columns = columnas

df = pd.concat([df1, df2, df3])
df = df[~df['etapa'].isin(['M', 'ML'])]

df['etapa'] = df['etapa'].astype('float')


## predecir el estrato
puede ser util para una empresa que vende feritizantes o una empresa que se dedique a habilitar espacios,
encontrar nuevos clientes en funcion de lo poco o mucho que usen las tierras para otros cultivos que no sean
el maiz, frijol o arroz, y dependiendo del departamento, enfocados a los estratos C y D.

tambien puede ser interesante hacer algun tipo de campaña de mejora de conocimientos de ciembras enfocada a los productores 
con estratos de tipo C y D, que no aprovechan del todo las tierras


In [4]:
scaler = StandardScaler()
escalados = scaler.fit_transform(df.select_dtypes('number'))
escalados = pd.DataFrame(escalados, columns=df.columns[df.dtypes != 'object'])


label_encoder = LabelEncoder().fit(df['estrato'])
y_encoded = label_encoder.transform(df['estrato'])

x_train, x_test, y_train, y_test = train_test_split(escalados.values, y_encoded, test_size=.2, random_state=10)

In [5]:
predictor_de_estrato = MLPRegressor(
    hidden_layer_sizes=(24, 12,), 
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=10
)

predictor_de_estrato.fit(x_train, y_train)

MLPRegressor(hidden_layer_sizes=(24, 12), max_iter=1000, random_state=10)

In [6]:
predictor_de_estrato.score(x_test, y_test)

0.8313915880371834

In [7]:
nuevo = pd.DataFrame({
    'departamento':[17],
    'municipio': [15], 
    'etapa': [1.], 
    'maiz_c': [0.3], 
    'frijol_c': [0.], 
    'arroz_c': [1.], 
    'otros_usos': [25.],
    'maiz_p': [1.], 
    'frijol_p': [1.], 
    'arroz_p': [1.]
})

x_pred =  scaler.transform(nuevo)
x_pred = pd.DataFrame(x_pred, columns=nuevo.columns)



# el arbol de decision dio un rango de [0, 25.7] y la red hizo una regrecion en el mismo intervalo 

print(f'codificacion de A B C D {set(y_encoded)}')

pred = predictor_de_estrato.predict(x_pred.values).round(1)
f'se predijo{pred} que equivale a una A -> 0 por ser el mas cercano'

codificacion de A B C D {np.int64(0), np.int64(1), np.int64(2), np.int64(3)}


'se predijo[-0.4] que equivale a una A -> 0 por ser el mas cercano'

## predecir frijol

In [8]:
scaler = StandardScaler().fit(df.select_dtypes('number'))
escalados = scaler.transform(df.select_dtypes('number'))
escalados = pd.DataFrame(escalados, columns=df.columns[df.dtypes != 'object'])

escalados['estrato'] = df['estrato'].values
escalados = pd.get_dummies(escalados, columns=['estrato'])
escalados.pop('frijol_p')
y = df['frijol_p'] 

x_train, x_test, y_train, y_test = train_test_split(escalados.values, y, test_size=.2, random_state=10)


In [9]:
predictor_de_frijol_p = MLPRegressor(
    hidden_layer_sizes=(24, 12, 6,), 
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=10
)

predictor_de_frijol_p.fit(x_train, y_train)

MLPRegressor(hidden_layer_sizes=(24, 12, 6), max_iter=1000, random_state=10)

In [10]:
predictor_de_frijol_p.score(x_test, y_test)

0.6891453733879409

In [11]:
nuevo = pd.DataFrame({
    'departamento':[17],
    'municipio': [5], 
    'etapa': [1.], 
    'maiz_c': [0.], 
    'frijol_c': [0.], 
    'arroz_c': [1.], 
    'otros_usos': [1.],
    'maiz_p': [1.], 
    'frijol_p': [1.], 
    'arroz_p': [1.],
    'estrato_A': [True],	
    'estrato_B': [False],	
    'estrato_C': [False],	
    'estrato_D': [False]
})

vals =  scaler.transform(nuevo.select_dtypes(exclude='bool'))
vals = pd.DataFrame(vals, columns=nuevo.columns[nuevo.dtypes != 'bool'])
x_pred = pd.concat([vals, nuevo.select_dtypes('bool')], axis=1)

x_pred.pop('frijol_p')
# el arbol de decision dio un rango de [0, 25.7] y la red hizo una regrecion en el mismo intervalo 
pred = predictor_de_frijol_p.predict(x_pred.values)
print(f'el arbol de decision dio un rango de [0, 25.7] y la red hizo una regrecion en el mismo intervalo  {pred}')

el arbol de decision dio un rango de [0, 25.7] y la red hizo una regrecion en el mismo intervalo  [0.05831325]
